## Prototype

In dit bestand is het prototype voor de brandweer opgezet. Om het prototype op te zetten zijn verschillende stappen uitgevoerd die per onderdeel worden uitgelicht. Hierin wordt uitgelicht welke keuzes er gemaakt zijn om de stappen uit te kunnen voeren.


#### Data preparatie
Vanuit de brandweer regio Rotterdam-Rijnmond zijn video's verkregen van vijf incidenten. Tijdens deze incidenten is er gevlogen met een drone om de videobeelden te kunnen maken. Hierin wordt van bovenaf gefilmd waarbij de locatie met omgeving en de brand in kaart wordt gebracht. Vorodat de data in het transfer learning model getraind kan worden is het nodig om de data te labelen. Om dit te kunnen doen wordt gebruik gemaakt van object detectie om zowel de objecten in de omgeving als de brand met bijbehorende rook te labelen. Hiervoor is gebruik gemaakt van het Yolo model versie 5.

Als eerst is het yolo model gedownload met de volgende code: pip install yolov5. Vervolgens zijn in de terminal de objecten rook en vuur getraind met de D-Fire dataset (Gaiasd, z.d.). De video's van de brandweer zijn vervolgens gedetecteer op de getrainde objecten vuur en rook. De video's waarin deze objecten gedetecteerd worden zijn vervolgens lokaal opgeslagen en met deze video's wordt vervolgens het oppervlak per boudning box bepaald. De eerste stap is hierin om per frame de grootte van de bounding boxes te bepalen. Dit is uitgevoerd moet onderstaande code. 

In [14]:
import torch
from PIL import Image
import cv2

# laden van het Yolov5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\cboxm/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-5-25 Python-3.10.6 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [15]:
# Het openen van het videobestand
video_path = 'cboxm/yolov5/runs/detect/exp11/DJI_20130118095545_0002_W.mp4'
video = cv2.VideoCapture(video_path)

In [17]:
# Opsplitsen van frames in de video
while True:
    # Het frame lezen
    ret, frame = video.read()

    # De loop wordt onderborken als de video is afgelopen
    if not ret:
        break

    # Omzetten naar een afbeelding
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Verwijzing naar afbeelding maken
    results = model(image)

    # Coordinaten van de bounding box
    bounding_boxes = results.pred[0].xyxy

    # Herhalen voor ieder object
    for box in bounding_boxes:
        x1, y1, x2, y2, conf, cls = box.tolist()

        # Printen van de bounding boxes met labels
        print(f"Frame: {frame_number}, Box: [{x1}, {y1}, {x2}, {y2}], Confidence: {conf}, Class: {cls}")

    # Het frame van de bounding boxes weergeven
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Video genereren
video.release()
cv2.destroyAllWindows()

#### Model voor labelen highlights

Naast het labelen van de highlights kan de video ook worden versneld om een verandering in de objecten vuur en rook te kunnen weergeven voor een Officier van Dienst. Met de volgende code is een video versneld.

In [18]:
from moviepy.editor import VideoFileClip
import moviepy.video.fx.all as vfx

# Ophalen video en definieren locatie versnelde video
video_path = 'runs/detect/exp11/DJI_20130118095545_0002_W.mp4'
video_out = 'runs/detect/exp11/DJI_20130118095545_0002_W_out.mp4'

# Importeren van de video in VideoFileClip
clip = VideoFileClip(video_path)
print("fps: {}".format(clip.fps))

# De FPS aanpassen
clip = clip.set_fps(clip.fps * 5)

# Versnellen van de video
final = clip.fx(vfx.speedx, 5)
print("fps: {}".format(final.fps))

# Opslaan van de video
final.write_videofile(video_out)

fps: 29.97002997002997
fps: 149.85014985014985
Moviepy - Building video runs/detect/exp11/DJI_20130118095545_0002_W_out.mp4.
Moviepy - Writing video runs/detect/exp11/DJI_20130118095545_0002_W_out.mp4



Moviepy - Done !
Moviepy - video ready runs/detect/exp11/DJI_20130118095545_0002_W_out.mp4


#### Model trainen

Het transfer learning model wordt getraind met het voorafgetraind ConvNet model. Het model bestaat uit een encoder en een decoder die ervoor zorgen dat voorspellingen op de videoframes gemaakt kunnen worden. In dit onderzoek wordt voorspelt of een frame een highlight is die meegenomen moet worden in de videosamenvatting. Door alle frames die voorspeld zijn als highlight achter elkaar te zetten ontstaat een videosamenvatting.